# Exercise 4
Write a discrete event simulation program for a blocking system, i.e. a system with m service units and no waiting room. The offered traffic A is the product of the mean arrival rate and the mean service time.

In [83]:
import numpy as np
from scipy.stats import t

def t_test_confidence_interval(data, alpha=0.05):
    # Sample size
    n = len(data)
    
    # Sample mean and standard deviation
    sample_mean = np.mean(data)
    sample_std = np.std(data, ddof=1)  # Use ddof=1 for unbiased estimate
    
    # Degrees of freedom
    df = n - 1
    
    # t critical value for two-tailed test
    t_critical = t.ppf(1 - alpha/2, df)
    
    # Margin of error
    margin_of_error = t_critical * (sample_std / np.sqrt(n))
    
    # Confidence interval bounds
    lower_bound = sample_mean - margin_of_error
    upper_bound = sample_mean + margin_of_error
    
    return (lower_bound, upper_bound)

## 1. 
The arrival process is modelled as a Poisson process. Report the fraction of blocked customers, and a confidence interval for this fraction. Choose the service time distribution as exponential. Parameters: m = 10, mean service time = 8 time units, mean time between customers = 1 time unit (corresponding to an offered traffic of 8 Erlang), 10 x 10.000 customers. This system is sufficiently simple such that the analytical solution is known. See the last slide for the solution. Verify your simulation program using this knowledge.

In [23]:
# Known Parameters:
m = 10 # number of service units
mst = 8 # mean service time, in time units
mtbc = 1 # mean time between customers, in time units
batches = 10 # number of batches
customers_per_batch = 10_000 # number of customers per batch

# The analytical solution
A = mst * mtbc  # Offered traffic A
Erlang_B = (A ** m / np.math.factorial(m)) / sum(A ** k / np.math.factorial(k) for k in range(m + 1))

In [85]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import poisson, norm
import scipy.stats as st 

# arrival times are poisson processes
# service times are exponential
def sim():
    at = np.cumsum(np.random.exponential(scale=mtbc, size=customers_per_batch)) # Poisson arrival times: CDF = P(Xi ≤ t) = 1 − e−λt
    st = np.random.exponential(scale=mst, size=customers_per_batch) # Exponential service times

    # Event queue for busy service units m
    service_busy_until = np.zeros(m)
    blocked_customers = 0

    # Iterate through each customer
    for i in range(customers_per_batch):
        
        # Check if any service unit is free
        if any(service_busy_until <= at[i]):
            
            # Find the first free service unit
            not_in_use = np.where(service_busy_until <= at[i])[0][0]
            service_busy_until[not_in_use] = at[i] + st[i]
        else:
            blocked_customers += 1
    
    return blocked_customers / customers_per_batch

blocked_frac = [sim () for batch in range(batches)]

# Calculate mean and standard deviation
mean_blocked_fraction = np.mean(blocked_frac)
std_blocked_fraction = np.std(blocked_frac, ddof=1)

# Calculate the 95% confidence interval
ci = st.t.interval(0.95, df=batches-1, loc=mean_blocked_fraction, scale=std_blocked_fraction / np.sqrt(batches))

print(f"Analytical solution (Erlang B formula): {Erlang_B}")
print(f"Mean fraction of blocked customers: {mean_blocked_fraction}")
print(f"Variance of fraction of blocked customers: {std_blocked_fraction**2}")
print(f"95% confidence interval: {ci}")

(0.11644106586518098, 0.12395893413481902)
Analytical solution (Erlang B formula): 0.12166106425295149
Mean fraction of blocked customers: 0.1202
Variance of fraction of blocked customers: 2.7611111111111164e-05
95% confidence interval: (0.11644106586518098, 0.12395893413481902)


## 2.
The arrival process is modelled as a renewal process using the same parameters as in Part 1 when possible. Report the fraction of blocked customers, and a confidence interval for this fraction for at least the following two cases:

- (a) Experiment with Erlang distributed inter arrival times. The Erlang distribution should have a mean of 1.
- (b) Hyper exponential inter arrival times. The parameters for the hyper exponential distribution should be p1 = 0.8, λ1 = 0.8333, p2 = 0.2, λ2 = 5.0.

In [69]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import poisson, norm

### 2 (a)

# arrival times are erlang distributed
# service times are exponential
def sim():
    at = np.cumsum(np.random.gamma(1, mtbc, customers_per_batch)) # Erlang arrival times
    st = np.random.exponential(scale=mst, size=customers_per_batch) # Exponential service times

    # Event queue for busy service units m
    service_busy_until = np.zeros(m)
    blocked_customers = 0

    # Iterate through each customer
    for i in range(customers_per_batch):
        
        # Check if any service unit is free
        if any(service_busy_until <= at[i]):
            
            # Find the first free service unit
            not_in_use = np.where(service_busy_until <= at[i])[0][0]
            service_busy_until[not_in_use] = at[i] + st[i]
        else:
            blocked_customers += 1
    
    return blocked_customers / customers_per_batch

blocked_frac = [sim () for batch in range(batches)]

# Calculate mean and standard deviation
mean_blocked_fraction = np.mean(blocked_frac)
std_blocked_fraction = np.std(blocked_frac, ddof=1)

# Calculate the 95% confidence interval
ci = st.t.interval(0.95, df=batches-1, loc=mean_blocked_fraction, scale=std_blocked_fraction / np.sqrt(batches))

print(f"Analytical solution (Erlang B formula): {Erlang_B}")
print(f"Mean fraction of blocked customers: {mean_blocked_fraction}")
print(f"Variance of fraction of blocked customers: {std_blocked_fraction**2}")
print(f"95% confidence interval: {ci}")

Analytical solution (Erlang B formula): 0.12166106425295149
Mean fraction of blocked customers: 0.1213
Variance of fraction of blocked customers: 5.326222222222221e-05
95% confidence interval: (0.11607925579307482, 0.1265207442069252)


In [65]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import poisson, norm

### 2 (b)

# arrival times have hyper exponential inter arrival times
# service times are exponential

def hyper_exponential(n):
    # Parameters
    p1 = 0.8
    lambda1 = 0.8333
    p2 = 0.2
    lambda2 = 5.0

    # Generate uniform random numbers
    U = np.random.uniform(size=n)
    
    # Determine which exponential distribution to sample from
    exp_distributions = np.where(U < p1, lambda1, lambda2)
    
    return np.random.exponential(scale=1/exp_distributions, size=n)
    

def sim():
    at = np.cumsum(hyper_exponential(customers_per_batch)) # Hyper exponential arrival times
    st = np.random.exponential(scale=mst, size=customers_per_batch) # Exponential service times

    # Event queue for busy service units m
    service_busy_until = np.zeros(m)
    blocked_customers = 0

    # Iterate through each customer
    for i in range(customers_per_batch):
        
        # Check if any service unit is free
        if any(service_busy_until <= at[i]):
            
            # Find the first free service unit
            not_in_use = np.where(service_busy_until <= at[i])[0][0]
            service_busy_until[not_in_use] = at[i] + st[i]
        else:
            blocked_customers += 1
    
    return blocked_customers / customers_per_batch

blocked_frac = [sim () for batch in range(batches)]

# Calculate mean and standard deviation
mean_blocked_fraction = np.mean(blocked_frac)
std_blocked_fraction = np.std(blocked_frac, ddof=1)

# Calculate the 95% confidence interval
ci = st.t.interval(0.95, df=batches-1, loc=mean_blocked_fraction, scale=std_blocked_fraction / np.sqrt(batches))

print(f"Mean fraction of blocked customers: {mean_blocked_fraction}")
print(f"Variance of fraction of blocked customers: {std_blocked_fraction**2}")
print(f"95% confidence interval: {ci}")

Mean fraction of blocked customers: 0.13562000000000002
Variance of fraction of blocked customers: 3.529955555555555e-05
95% confidence interval: (0.13136981931982203, 0.139870180680178)


## 3.
The arrival process is again a Poisson process like in Part 1. Experiment with different service time distributions with the same mean service time and m as in Part 1 and Part 2.
- (a) Constant service time
- (b) Pareto distributed service times with at least k = 1.05 and k = 2.05.
- (c) Choose one or two other distributions.

In [71]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import poisson, norm

### 3 (a)

# New Parameters:
cst = 8 # constant service time, in time units

# arrival times are poisson processes
# service times are constant
def sim():
    at = np.cumsum(np.random.exponential(scale=mtbc, size=customers_per_batch)) # Poisson arrival times: CDF = P(Xi ≤ t) = 1 − e−λt
    st = cst # Constant service times
    
    # Event queue for busy service units m
    service_busy_until = np.zeros(m)
    blocked_customers = 0

    # Iterate through each customer
    for i in range(customers_per_batch):
        
        # Check if any service unit is free
        if any(service_busy_until <= at[i]):
            
            # Find the first free service unit
            not_in_use = np.where(service_busy_until <= at[i])[0][0]
            service_busy_until[not_in_use] = at[i] + st
        else:
            blocked_customers += 1
    
    return blocked_customers / customers_per_batch

blocked_frac = [sim () for batch in range(batches)]

# Calculate mean and standard deviation
mean_blocked_fraction = np.mean(blocked_frac)
std_blocked_fraction = np.std(blocked_frac, ddof=1)

# Calculate the 95% confidence interval
ci = st.t.interval(0.95, df=batches-1, loc=mean_blocked_fraction, scale=std_blocked_fraction / np.sqrt(batches))

print(f"Analytical solution (Erlang B formula): {Erlang_B}")
print(f"Mean fraction of blocked customers: {mean_blocked_fraction}")
print(f"Variance of fraction of blocked customers: {std_blocked_fraction**2}")
print(f"95% confidence interval: {ci}")

Analytical solution (Erlang B formula): 0.12166106425295149
Mean fraction of blocked customers: 0.12057000000000002
Variance of fraction of blocked customers: 1.1737888888888908e-05
95% confidence interval: (0.11811914413449714, 0.1230208558655029)


In [73]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import poisson, norm

### 3 (b)

# New Parameters:
beta = 1 # Just arbitrarily chosen

# arrival times are poisson processes
# service times are pareto distributed
def sim(k):
    at = np.cumsum(np.random.exponential(scale=mtbc, size=customers_per_batch)) # Poisson arrival times: CDF = P(Xi ≤ t) = 1 − e−λt
    st = (np.random.pareto(k, customers_per_batch) + beta) * (mst * (k - 1) / k) # Pareto service times

    # Event queue for busy service units m
    service_busy_until = np.zeros(m)
    blocked_customers = 0

    # Iterate through each customer
    for i in range(customers_per_batch):
        
        # Check if any service unit is free
        if any(service_busy_until <= at[i]):
            
            # Find the first free service unit
            not_in_use = np.where(service_busy_until <= at[i])[0][0]
            service_busy_until[not_in_use] = at[i] + st[i]
        else:
            blocked_customers += 1
    
    return blocked_customers / customers_per_batch

# k=1.05
blocked_frac_k_1_05 = [sim(k=1.05) for batch in range(batches)]

# Calculate mean and standard deviation
mean_blocked_fraction = np.mean(blocked_frac_k_1_05)
std_blocked_fraction = np.std(blocked_frac_k_1_05, ddof=1)

# Calculate the 95% confidence interval
ci = st.t.interval(0.95, df=batches-1, loc=mean_blocked_fraction, scale=std_blocked_fraction/np.sqrt(batches))

print(f"Analytical solution (Erlang B formula): {Erlang_B}\n")

print("k=1.05")
print(f"Mean fraction of blocked customers: {mean_blocked_fraction}")
print(f"Variance of fraction of blocked customers: {std_blocked_fraction**2}")
print(f"95% confidence interval: {ci}\n")

# k=2.05
blocked_frac_k_2_05 = [sim(k=2.05) for batch in range(batches)]

# Calculate mean and standard deviation
mean_blocked_fraction = np.mean(blocked_frac_k_2_05)
std_blocked_fraction = np.std(blocked_frac_k_2_05, ddof=1)

# Calculate the 95% confidence interval
ci = st.t.interval(0.95, df=batches-1, loc=mean_blocked_fraction, scale=std_blocked_fraction/np.sqrt(batches))

print("k=2.05")
print(f"Mean fraction of blocked customers: {mean_blocked_fraction}")
print(f"Variance of fraction of blocked customers: {std_blocked_fraction**2}")
print(f"95% confidence interval: {ci}")

Analytical solution (Erlang B formula): 0.12166106425295149

k=1.05
Mean fraction of blocked customers: 0.0023
Variance of fraction of blocked customers: 1.623333333333333e-05
95% confidence interval: (-0.000582216681129278, 0.005182216681129278)

k=2.05
Mean fraction of blocked customers: 0.12086999999999999
Variance of fraction of blocked customers: 5.988677777777779e-05
95% confidence interval: (0.11533409986155688, 0.1264059001384431)


In [75]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import poisson, norm

### 3 (c) - Normally distributed service times

# New Parameters:
mean = 8  # Mean service time
std = 2  # Standard deviation of service time

# arrival times are poisson processes
# service times are pareto distributed
def sim():
    at = np.cumsum(np.random.exponential(scale=mtbc, size=customers_per_batch)) # Poisson arrival times: CDF = P(Xi ≤ t) = 1 − e−λt
    st = np.random.normal(mean, std, customers_per_batch) # Normal service times

    # Event queue for busy service units m
    service_busy_until = np.zeros(m)
    blocked_customers = 0

    # Iterate through each customer
    for i in range(customers_per_batch):
        
        # Check if any service unit is free
        if any(service_busy_until <= at[i]):
            
            # Find the first free service unit
            not_in_use = np.where(service_busy_until <= at[i])[0][0]
            service_busy_until[not_in_use] = at[i] + st[i]
        else:
            blocked_customers += 1
    
    return blocked_customers / customers_per_batch

blocked_frac = [sim() for batch in range(batches)]

# Calculate mean and standard deviation
mean_blocked_fraction = np.mean(blocked_frac)
std_blocked_fraction = np.std(blocked_frac, ddof=1)

# Calculate the 95% confidence interval
ci = st.t.interval(0.95, df=batches-1, loc=mean_blocked_fraction, scale=std_blocked_fraction/np.sqrt(batches))

print(f"Analytical solution (Erlang B formula): {Erlang_B}")
print(f"Mean fraction of blocked customers: {mean_blocked_fraction}")
print(f"Variance of fraction of blocked customers: {std_blocked_fraction**2}")
print(f"95% confidence interval: {ci}")

Analytical solution (Erlang B formula): 0.12166106425295149
Mean fraction of blocked customers: 0.12048
Variance of fraction of blocked customers: 4.074844444444439e-05
95% confidence interval: (0.11591355436800925, 0.12504644563199077)


## 4.  Compare confidence intervals for Parts 1, 2, and 3 then interpret and explain differences if any.
We see that...